In [1]:
import requests
import pandas as pd
import json

Let's collect the relevant data for 'feed000000000000000000000000cafe' key from https://api.eu.navixy.com 

To get the trackers' states, we first need to obtain all the tracker_ids available for the key.

In [2]:
url = "https://api.eu.navixy.com/v2/tracker/list"

headers = {
    # Added when you pass json=
    # 'Content-Type': 'application/json',
}

json_data = {
    'hash': 'feed000000000000000000000000cafe',
}
response = requests.post(url, headers=headers, json=json_data)
tracker_list = response.json()
tracker = pd.json_normalize(tracker_list['list'])
tracker.to_csv('../data/tracker.csv', index=False)

In [3]:
tracker.head(2)

,id,label,group_id,tag_bindings,clone,source.status_listing_id,source.id,source.device_id,source.blocked,source.creation_date,source.tariff_id,source.model,source.tariff_end_date,source.phone
0,877766,Steve (MAN),105448,[],False,NaN,545138,979914665171717,False,2021-08-09,1352,atoldrive5,2024-09-01,None
1,877767,John (Scania),105448,[],False,NaN,545139,693299870436055,False,2021-08-09,1352,atoldrive5,2024-09-01,None


We may only need the 'label' for the dashboard, so let's save the .csv with the unneeded columns removed.

In [37]:
tracker[['id', 'label']].rename(columns={'id':'tracker_id'}).to_csv('data/tracker_label.csv', index=False)

In [4]:
url = "https://api.eu.navixy.com/v2/tracker/get_states"

json_data = {
    'hash': 'feed000000000000000000000000cafe',
    'trackers': tracker.id.to_list(),
}
response = requests.post(url, headers=headers, json=json_data)
trackers_states = response.json()
df = pd.DataFrame.from_dict(trackers_states['states'], orient='index')
gps = pd.json_normalize(df['gps']).add_prefix('gps.')
gsm = pd.json_normalize(df['gsm']).add_prefix('gsm.')
df = pd.concat([df.reset_index().rename(columns={'index': 'id'}), gps, gsm], axis=1).drop(columns={"gps", "gsm"})
df.to_csv('../data/trackers_states.csv', index=False)

In [5]:
with pd.option_context("display.max_columns", 100, 'display.max_colwidth', 100):
    display(df.head(2))

,id,source_id,connection_status,movement_status,last_update,battery_level,battery_update,inputs,inputs_update,outputs,outputs_update,actual_track_update,additional,gps.updated,gps.signal_level,gps.heading,gps.speed,gps.alt,gps.location.lat,gps.location.lng,gsm.updated,gsm.signal_level,gsm.network_name,gsm.roaming
0,3036047,10230656,offline,parked,2024-07-23 04:11:54,NaN,None,[False],2024-07-23 04:11:54,[],2024-07-23 04:11:54,2024-07-22 23:02:12,NaN,2024-07-23 04:09:12,100,45,0,0,38.133215,-85.768903,None,0,,None
1,3036045,10230654,offline,parked,2024-07-23 04:12:52,NaN,None,"[False, False, False, False, False, False]",2024-07-23 04:12:52,"[False, False, False]",2024-07-23 04:12:52,2024-07-22 21:41:39,"{'event_code': {'value': '0', 'updated': '2024-07-23 04:09:50'}}",2024-07-23 04:09:50,100,45,0,0,30.328325,-97.720808,None,0,,False


We can use the following attributes for the dashboard: connection_status, movement_status, last_update, gps.location.lat, and gps.location.lng (latitude and longitude). Let's look more closely at them.

In [6]:
df.movement_status.value_counts()

movement_status
parked     10
moving      3
stopped     3
Name: count, dtype: int64

In [7]:
df.connection_status.value_counts()

connection_status
offline    15
active      1
Name: count, dtype: int64

In [8]:
pd.to_datetime(df.last_update).dt.date.value_counts()

last_update
2024-07-23    15
2024-08-02     1
Name: count, dtype: int64

Let's save only the columns needed for vizualization

In [36]:
df.rename(columns={'id':'tracker_id'})[['tracker_id', 'connection_status', 'movement_status', 'last_update', 'gps.speed', 'gps.location.lat', 'gps.location.lng']].to_csv('data/trackers_states_viz.csv', index=False)

I tried to collect information on drivers, vehicles, fuel level, sensor readings, and inputs. However, most of the data is partial and therefore not valuable for visualization.

In [9]:
url = "https://api.eu.navixy.com/v2/employee/list"

json_data = {
    'hash': 'feed000000000000000000000000cafe',
}
response = requests.post(url, headers=headers, json=json_data)
employee = pd.json_normalize(response.json()['list'])
employee.to_csv('../data/employee.csv', index=False)

In [10]:
employee.head()

,id,avatar_file_name,tracker_id,first_name,middle_name,last_name,email,phone,driver_license_number,driver_license_cats,driver_license_issue_date,driver_license_valid_till,hardware_key,department_id,location.lat,location.lng,location.address,location.radius,personnel_number,icon_id
0,155,e11dca3227e9169edaee9881607346ab.png,NaN,Rahul,,Adenauer,crhis.adenauer@navixy.com,41578994110,YTS0823139B,,None,2027-08-10,,195,50.937229,7.080688,"Lehmbacher Weg 130A, 51109 Köln, Germany",150.0,NaN,NaN
1,156,1937683bd6d62d30b311a45570d3f01d.jpeg,NaN,Michael,,Zumthor,michael.zumthor@navixy.com,41578112457,JYR9237081S,,None,2024-07-25,,196,50.912125,6.688614,"Erftstraße 174, 50170 Kerpen, Germany",150.0,NaN,NaN
2,2183,a755d6d8d01351c653502fb82f83b33b.jpeg,NaN,Samantha Beckett,,,ann.beckett@navixy.com,4564234671,EU5903490TR,,None,None,None,194,NaN,NaN,NaN,NaN,,NaN
3,2184,9c41cd706e7ebba3322c0d994707b5b4.png,NaN,Paul,,Anderson,paul.anderson@navixy.com,343234555434,,,None,None,,195,NaN,NaN,NaN,NaN,NaN,NaN
4,70333,NaN,877767.0,John Smith,,,,,,,None,None,123,195,NaN,NaN,NaN,NaN,,5.0


In [11]:
url = "https://api.eu.navixy.com/v2/vehicle/list"

json_data = {
    'hash': 'feed000000000000000000000000cafe',
}
response = requests.post(url, headers=headers, json=json_data)
vehicle = pd.json_normalize(response.json()['list'])
vehicle.to_csv('../data/vehicle.csv', index=False)

In [12]:
vehicle.head()

,id,avatar_file_name,icon_color,tracker_id,tracker_label,label,max_speed,model,type,subtype,...,norm_avg_fuel_consumption,fuel_tank_volume,fuel_cost,wheel_arrangement,tyre_size,tyres_number,liability_insurance_policy_number,liability_insurance_valid_till,free_insurance_policy_number,free_insurance_valid_till
0,1133,38391c000a8a72dafad10d9a66a79915.jpeg,1E96DC,877766.0,Steve (MAN),Man,85.0,MAN TGA 33.480,truck,tractor,...,17.0,400.0,NaN,6x4,215/75 R17.5,4.0,,None,,None
1,1134,efcf72396a9a8b17b947e6c3dc444bdc.jpeg,1E96DC,NaN,NaN,Ford,90.0,65117-3010,truck,awning,...,28.0,350.0,NaN,6x2,10.00 R20,10.0,,None,,None
2,96611,2349252afd024c9330c5e0c8fc59b18a.jpeg,1E96DC,877767.0,John (Scania),Scania,30.0,,truck,NaN,...,10.0,10.0,1.0,,,NaN,,None,,None


In [13]:
url = "https://api.eu.navixy.com/v2/tracker/get_fuel"

fuel_states = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
    }
    response = requests.post(url, headers=headers, json=json_data)
    df = pd.json_normalize(response.json())
    df = df.explode('inputs')
    if pd.isna(df.loc[0, 'inputs']) == False:
        df = pd.concat([df, df['inputs'].apply(pd.Series)], axis=1)
    df['tracker_id'] = tracker_id
    fuel_states.append(df)

df = pd.concat(fuel_states, axis=0, ignore_index=True).drop(columns=['inputs', 'user_time'])
df.to_csv('../data/fuel_states.csv', index=False)

/tmp/ipykernel_25549/689927789.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(fuel_states, axis=0, ignore_index=True).drop(columns=['inputs', 'user_time'])


In [14]:
df.head()

,update_time,success,label,units,name,type,min_value,max_value,value,units_type,converted_units_type,converted_value,tracker_id
0,2022-07-13 08:26:01,True,Fuel,,lls_level_1,fuel,11.0,15.0,15.00,litre,None,None,877766
1,2022-10-05 03:49:09,True,LLS: Level #1,,lls_level_1,fuel,0.0,10.0,797.17,litre,None,None,877767
2,2022-06-15 20:10:37,True,LLS: Level #1,,lls_level_1,fuel,NaN,NaN,227.40,litre,None,None,877768
3,None,True,OBD: Fuel,,obd_fuel,fuel,NaN,NaN,NaN,percent,None,None,3036043
4,None,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3036045


In [15]:
url = "https://api.eu.navixy.com/v2/tracker/get_diagnostics"

diagnostics = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
    }
    response = requests.post(url, headers=headers, json=json_data)
    diag = response.json()

    df = pd.json_normalize(diag)
    df = df.explode('inputs')
    if pd.isna(df.loc[0, 'inputs']) == False:
        df = pd.concat([df, df['inputs'].apply(pd.Series)], axis=1)
    df['tracker_id'] = tracker_id
    diagnostics.append(df)

df = pd.concat(diagnostics, axis=0, ignore_index=True).drop(columns=['inputs', 'user_time'])
df.to_csv('../data/diagnostics.csv', index=False)

In [16]:
url = "https://api.eu.navixy.com/v2/tracker/get_inputs"

inputs = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
    }
    response = requests.post(url, headers=headers, json=json_data)
    inp = response.json()

    df = pd.json_normalize(inp)
    df = df.explode('states')
    if pd.isna(df.loc[0, 'states']) == False:
        df = pd.concat([df, df['states'].apply(pd.Series)], axis=1)
    df['tracker_id'] = tracker_id
    inputs.append(df)

df = pd.concat(inputs, axis=0, ignore_index=True).drop(columns=['states', 'user_time'])
df.to_csv('../data/inputs.csv', index=False)

In [17]:
df.head()

,update_time,inputs,success,tracker_id,type,name,status,input_number
0,2024-07-23 04:10:01,"[False, False, False]",True,877766,NaN,NaN,NaN,NaN
1,2024-07-23 04:09:59,"[False, False, False]",True,877767,NaN,NaN,NaN,NaN
2,2024-07-23 04:10:01,"[False, False, False]",True,877768,NaN,NaN,NaN,NaN
3,2024-07-23 04:11:54,"[False, False, False]",True,3036043,ignition,Ignition,False,1.0
4,2024-07-23 04:12:52,"[False, False, False, False, False, False]",True,3036045,ignition,Ignition,False,1.0


In [21]:
url = "https://api.eu.navixy.com/v2/tracker/get_readings"

readings = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
    }
    response = requests.post(url, headers=headers, json=json_data)
    read = response.json()

    df = pd.json_normalize(read)
    df = df.explode('inputs').reset_index(drop=True)
    if pd.isna(df.loc[0, 'inputs']) == False:
        df = pd.concat([df, df['inputs'].apply(pd.Series)], axis=1)
    df['tracker_id'] = tracker_id
    readings.append(df)

df = pd.concat(readings, axis=0, ignore_index=True).drop(columns=['inputs', 'user_time'])
df.to_csv('../data/readings.csv', index=False)

In [22]:
df.head()

,update_time,success,tracker_id,label,units,name,type,value,units_type,converted_units_type,converted_value
0,None,True,877766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,True,877767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,True,877768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-07-23 04:09:21,True,3036043,Board voltage,,board_voltage,power,12.4,volt,None,None
4,2024-07-23 04:09:50,True,3036045,Board voltage,,board_voltage,power,24.9,volt,None,None


I gathered the last points of the trackers located by GPS

In [18]:
url = "https://api.eu.navixy.com/v2/tracker/get_last_gps_point"

# headers = {
#     'Content-Type': 'application/json',
# }

gps_list = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
    }
    response = requests.post(url, headers=headers, json=json_data)
    df = pd.json_normalize(response.json())
    df['tracker_id'] = tracker_id
    gps_list.append(df)

df = pd.concat(gps_list, axis=0, ignore_index=True)
df.to_csv('../data/last_gps_point.csv', index=False)

In [19]:
df.head(2)

,success,value.lat,value.lng,value.satellites,value.get_time,value.heading,value.speed,tracker_id,value.alt
0,True,40.663340,-73.818610,10,2024-07-23 04:10:01,99,52,877766,NaN
1,True,43.153782,-78.519517,10,2024-07-23 04:09:59,336,49,877767,NaN


However, the GPS information from this endpoint is exactly the same we previously retrieved from "tracker/get_states"

In [20]:
loc_compare = df[['tracker_id', 'value.lat', 'value.lng']].merge(pd.read_csv('data/trackers_states.csv')[['id', 'gps.location.lat', 'gps.location.lng']], left_on='tracker_id', right_on='id')

In [33]:
loc_compare.head(2)

,tracker_id,value.lat,value.lng,id,gps.location.lat,gps.location.lng
0,877766,40.663340,-73.818610,877766,40.663340,-73.818610
1,877767,43.153782,-78.519517,877767,43.153782,-78.519517


In [31]:
loc_compare[loc_compare['value.lat'] != loc_compare['gps.location.lat']]

,tracker_id,value.lat,value.lng,id,gps.location.lat,gps.location.lng


In [32]:
loc_compare[loc_compare['value.lng'] != loc_compare['gps.location.lng']]

,tracker_id,value.lat,value.lng,id,gps.location.lat,gps.location.lng


The most accurate and comprehensive additional information available on the trackers is their mileage.

In [23]:
url = "https://api.eu.navixy.com/v2/tracker/stats/mileage/read"
 
json_data = {
    'hash': 'feed000000000000000000000000cafe',
    'trackers': tracker.id.to_list(),
    "from": "2024-06-24 00:00:00", 
    "to": "2024-07-24 00:00:00",
}
response = requests.post(url, headers=headers, json=json_data)
dfm = pd.DataFrame.from_dict(response.json()['result'], orient='index')

In [24]:
with pd.option_context("display.max_columns", 100, 'display.max_colwidth', 100):
    display(dfm.head(2))

,2024-06-24,2024-06-25,2024-06-26,2024-06-27,2024-06-28,2024-06-29,2024-06-30,2024-07-01,2024-07-02,2024-07-03,2024-07-04,2024-07-05,2024-07-06,2024-07-07,2024-07-08,2024-07-09,2024-07-10,2024-07-11,2024-07-12,2024-07-13,2024-07-14,2024-07-15,2024-07-16,2024-07-17,2024-07-18,2024-07-19,2024-07-20,2024-07-21,2024-07-22,2024-07-23
3036047,{'mileage': 249.8},{'mileage': 276.66},{'mileage': 252.43},{'mileage': 252.63},{'mileage': 246.38},{'mileage': 257.19},{'mileage': 246.0},{'mileage': 256.98},{'mileage': 269.11},{'mileage': 260.5},{'mileage': 260.71},{'mileage': 260.72},{'mileage': 264.85},{'mileage': 269.94},{'mileage': 265.6},{'mileage': 0.0},None,None,{'mileage': 264.64},{'mileage': 248.17},{'mileage': 242.76},{'mileage': 245.23},{'mileage': 264.37},{'mileage': 0.0},{'mileage': 259.52},{'mileage': 237.19},{'mileage': 241.39},{'mileage': 250.0},{'mileage': 275.38},{'mileage': 0.0}
3036045,{'mileage': 301.0},{'mileage': 902.96},{'mileage': 300.99},{'mileage': 602.05},{'mileage': 435.6},{'mileage': 467.2},{'mileage': 601.98},{'mileage': 301.03},{'mileage': 902.89},{'mileage': 301.01},{'mileage': 602.0},{'mileage': 451.0},{'mileage': 451.96},{'mileage': 602.02},{'mileage': 343.47},{'mileage': 244.16},None,None,{'mileage': 575.53},{'mileage': 601.89},{'mileage': 568.71},{'mileage': 334.1},{'mileage': 549.5},{'mileage': 301.02},{'mileage': 601.9},{'mileage': 602.05},{'mileage': 526.92},{'mileage': 376.18},{'mileage': 602.07},{'mileage': 0.0}


In [25]:
def get_mileage(x) -> float:
    if x is not None:
        for k,v in x.items():
            return v
    return None
dfm = dfm.map(lambda x: get_mileage(x)).reset_index().rename(columns={'index':'tracker_id'})
dfm = dfm.melt(id_vars=["tracker_id"], var_name="date").rename(columns={'value':'mileage'})
dfm.to_csv('../data/mileage.csv', index=False)
dfm.groupby(['tracker_id'])['mileage'].sum()

tracker_id
1228833       32.02
3036043     7270.93
3036045    13451.19
3036047     6418.15
3036049     6803.03
3036056     8534.67
3036057    26430.41
3036059     4177.44
3036060     5779.42
3036068     9453.07
3036069    22617.78
3036070     4333.01
3036071     5860.07
877766     19053.84
877767     21651.99
877768     20383.32
Name: mileage, dtype: float64

In [26]:
dfm.tracker_id.nunique()

16

Finally, although track data is available for only 14 out of 16 trackers, it allows for a detailed view of tracker activity for the month preceding the day of the last update (since all but one of the trackers had their last update on 2024-07-23, I chose that date). This includes the number of tracks per day, duration, distance traveled (to be compared with mileage), idle time, and average speed.

In [27]:
url = "https://api.eu.navixy.com/v2/track/list"

track_lists = []

for tracker_id in tracker.id.to_list(): 
    json_data = {
        'hash': 'feed000000000000000000000000cafe',
        'tracker_id': tracker_id,
        "from": "2024-06-24 00:00:00", 
        "to": "2024-07-24 00:00:00",
    }
    response = requests.post(url, headers=headers, json=json_data)
    df = pd.json_normalize(response.json()['list'])
    df['tracker_id'] = tracker_id
    track_lists.append(df)

df = pd.concat(track_lists, axis=0, ignore_index=True)
df.to_csv('../data/track.csv', index=False)

In [28]:
df.tracker_id.nunique()

14

Let's identify which trackers have no available tracks.

In [29]:
set(tracker.id.to_list()).difference(set(df.tracker_id.to_list()))

{1228833, 3036059}